In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from __future__ import division
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
sys.path.append('/data/mfmtk-utils/')
from mfmtkutils import *

In [2]:
zoo_info = np.loadtxt('/data/classifi2/EFIGI_sdss_mfmtk_ttype_zoo.csv',
                       delimiter=',', usecols=[15, 44], dtype=str).T
zoo_info[1]

array(['', '', '', ..., 'S', 'S', ''], 
      dtype='|S10')

In [4]:
data = catalog(path='/data/catalogs/1024_r.mfmtk')
galaxies = data.raw_catalog[0]
params = ['C1', 'A3', 'S3', 'H', 'sigma_psi']
data = data.param_selection(params)
data

array([[ 3.27228,  0.89891,  0.71036,  0.36193,  0.18204],
       [ 4.43201,  0.71831,  0.44517,  0.23225,  0.1308 ],
       [ 1.89655,  0.81395,  0.64167,  0.28105,  0.6757 ],
       ..., 
       [ 3.00056,  0.82209,  0.62808,  0.3271 ,  0.59636],
       [ 2.92652,  0.97244,  0.889  ,  0.49111,  0.35268],
       [ 2.66638,  0.8516 ,  0.6795 ,  0.57945,  0.60126]])

In [49]:
classes = np.array([zoo_info[1][np.where(zoo_info[0] == val)[0][0]] for i, val in enumerate(galaxies) if val in set(zoo_info[0])])
tipo = np.zeros(galaxies.shape, dtype='int8')
tipo[np.where(classes == 'U')] = 3
tipo[np.where(classes == '')] = 3
tipo[np.where(classes == 'S')] = 0
tipo[np.where(classes == 'E')] = 1
valid = np.where(tipo < 2)

In [61]:
NE = np.size(tipo[np.where(tipo == 1)])
NS = np.size(tipo[np.where(tipo == 0)])
priors = np.array([NS/(NE+NS), NE/(NE+NS)])
priors, NE, NS, NE+NS

(array([ 0.18384257,  0.81615743]), 1576, 355, 1931)

In [62]:
lda = LDA(solver='lsqr', priors=priors, store_covariance=1)
lda.fit(data[valid], tipo[valid])
predictions = lda.predict(data)
predictions.sum()

4161

In [29]:
data.shape, data[valid].shape, tipo[valid].sum()

((4161, 5), (3786, 5), 297)